In [1]:
import definitions as df
import data_manipulation as dm
import sql_def as sql
import os
import numpy as np
import matplotlib.pyplot as plt
import plots
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
import math
import torch.nn.functional as F

In [2]:
total_size = 200
max_x = 1000
max_y = 1000
min_x = 0
min_y = 0
track_len = 10

trajectories = dm.create_synthetic(total_size, max_x, max_y, min_x, min_y, track_len)
print(trajectories)

[[(599.0, 425.0), (604.991777, 425.314016), (610.983554, 425.628031), (616.975332, 425.942047), (622.967109, 426.256063), (628.958886, 426.570079), (634.950663, 426.884094), (640.94244, 427.19811), (646.934218, 427.512126), (652.925995, 427.826142)], [(520.0, 98.0), (520.0, 102.0), (520.0, 106.0), (520.0, 110.0), (520.0, 114.0), (520.0, 118.0), (520.0, 122.0), (520.0, 126.0), (520.0, 130.0), (520.0, 134.0)], [(77, 362), (87, 362), (97, 362), (107, 362), (117, 362), (127, 362), (137, 362), (147, 362), (157, 362), (167, 362)], [(587.0, 170.0), (589.344387, 164.476971), (591.688774, 158.953942), (594.03316, 153.430913), (596.377547, 147.907884), (598.721934, 142.384854), (601.066321, 136.861825), (603.410707, 131.338796), (605.755094, 125.815767), (608.099481, 120.292738)], [(627, 356), (632, 356), (637, 356), (642, 356), (647, 356), (652, 356), (657, 356), (662, 356), (667, 356), (672, 356)], [(8, 97), (17, 97), (26, 97), (35, 97), (44, 97), (53, 97), (62, 97), (71, 97), (80, 97), (89, 9

In [3]:
norm_all_data = df.normalize_data(trajectories)
train, test = df.split_test_train(norm_all_data)
train_data, train_masked_point = df.mask_point_at_index(train, 6)
test_data, test_masked_point = df.mask_point_at_index(test, 6)

# print(len(train_data))
# print(len(test_masked_point))

In [4]:
class TrajectoryDataset(Dataset):
    def __init__(self, trajectories, mask_index):
        self.trajectories = trajectories
        self.mask_index = mask_index

    def __len__(self):
        return len(self.trajectories)

    def __getitem__(self, idx):
        trajectory = np.array(self.trajectories[idx])
        masked_trajectory = np.copy(trajectory)
        target = masked_trajectory[self.mask_index].copy()
        masked_trajectory[self.mask_index] = None

        return torch.tensor(masked_trajectory, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)

dataset = TrajectoryDataset(train, mask_index=5)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
# print(dataset.trajectories)

In [5]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, max_len, dropout=0.1):
        super().__init__()
        self.input_embedding = nn.Linear(input_dim, model_dim)

        self.positional_encoding = nn.Parameter(torch.randn(max_len, model_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)

        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.output_layer = nn.Linear(model_dim, input_dim)

    def forward(self, x):
        x = self.input_embedding(x) + self.positional_encoding
        x = self.transformer_encoder(x)
        return self.output_layer(x[:, 5, :])

In [6]:
model = TransformerModel(input_dim=2, model_dim=128, num_heads=4, num_layers=2, max_len=10)

c:\Users\alito\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
import sys
import os

class SuppressPrint:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(10):
    model.train()
    total_loss = 0
    for masked_trajectories, targets in dataloader:
        optimizer.zero_grad()
        with SuppressPrint():
            predictions = model(masked_trajectories)
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan


In [ ]:
print(7^(-1) % 59273400)

In [ ]:
7**2

In [ ]:
17385377**7 % 59288951

In [ ]:
(37928070**59273392) % 59288951